In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
import wrangle
import warnings
warnings.filterwarnings("ignore")
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from pydataset import data
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
df = pd.read_csv('final_df.csv',index_col=[0])
columns_list = df.select_dtypes(exclude=['object']).columns.to_list()

list_to_remove = ['conf_champ','avg_att_air_yards','avg_air_yard_diff','time_to_throw','air_yds_to_sticks','won_superbowl',
'avg_comp_air_yds','superbowl','year']

for i in list_to_remove:
    if i in columns_list:
        columns_list.remove(i)

        
target = columns_list.pop(len(columns_list) -1 )

In [9]:
X_train

,aggressive,lon_comp_air_dist,pass_att,total_yds,td,int,pass_rating,completion_pct,exp_completion_pct,completion_pct_abv_exp,first_round,div_round
name,,,,,,,,,,,,
Tom Brady,15.2,50.9,613.0,4057.0,24.0,8.0,88.0,60.8,64.0,-3.1,1.0,0.0
Matt Ryan,16.3,52.0,560.0,3968.0,20.0,12.0,90.4,67.0,66.0,0.9,0.0,0.0
Trevor Siemian,20.4,57.3,486.0,3401.0,18.0,10.0,84.6,59.5,57.5,2.0,0.0,0.0
Matt Barkley,21.8,47.7,216.0,1611.0,8.0,14.0,68.3,59.7,58.7,1.0,0.0,0.0
Jalen Hurts,14.2,50.7,148.0,1061.0,6.0,4.0,77.6,52.0,55.5,-3.4,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Drew Stanton,20.1,47.0,159.0,894.0,6.0,5.0,66.4,49.7,59.1,-9.4,0.0,0.0
Davis Mills,15.2,52.9,479.0,3118.0,17.0,15.0,78.8,61.0,64.2,-3.2,0.0,0.0
Andy Dalton,15.0,48.3,333.0,2170.0,14.0,8.0,87.3,64.9,64.9,0.0,0.0,0.0


In [20]:
X = df[columns_list]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=123)
parameters_lars = {
    'alpha':range(1,21),
    'fit_intercept':[True, False],
    'verbose': [True, False]
}

parameters_linear = {'n_jobs':range(1,21)
                }

parameters_glm = {
    'link':['auto', 'identity', 'log'],
    'alpha':range(1,21),
}

lars = LassoLars(random_state =123)
Linear_regression1 = LinearRegression()
glm = TweedieRegressor(power=2, alpha=0)


the_parameters = [parameters_lars, parameters_linear, parameters_glm]


models = ['lars','Linear_regression1','glm']
master_df = pd.DataFrame()
for number, tree in enumerate([lars, Linear_regression1, glm]):
    grid = GridSearchCV(tree, the_parameters[number], cv=5, scoring = 'neg_root_mean_squared_error')
    grid.fit(X_train, y_train)
    
    for p, score in zip(grid.cv_results_['params'], grid.cv_results_['mean_test_score']):
        p['score'] = abs(score)
        p['model'] = models[number]
    new_df = pd.DataFrame(pd.DataFrame(grid.cv_results_['params']).sort_values('score', ascending=True))
    master_df = pd.concat([master_df, new_df])

In [21]:
master_df

,alpha,fit_intercept,verbose,score,model,n_jobs,link
79,20.0,False,False,6.128393,lars,NaN,NaN
78,20.0,False,True,6.128393,lars,NaN,NaN
75,19.0,False,False,6.129814,lars,NaN,NaN
74,19.0,False,True,6.129814,lars,NaN,NaN
70,18.0,False,True,6.131291,lars,NaN,NaN
71,18.0,False,False,6.131291,lars,NaN,NaN
66,17.0,False,True,6.132822,lars,NaN,NaN
67,17.0,False,False,6.132822,lars,NaN,NaN
76,20.0,True,True,6.133877,lars,NaN,NaN
77,20.0,True,False,6.133877,lars,NaN,NaN


In [3]:
df = pd.read_csv('2022.csv',index_col=[0])

In [5]:
df['year'] = '2022'

In [11]:
df =df.drop(columns=['Pos'])

In [12]:
df

,Player,Tm,Age,G,GS,QBrec,Cmp,Att,Cmp%,Yds,...,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,Player-additional,year
Rk,,,,,,,,,,,,,,,,,,,,,
1,Patrick Mahomes*+,KAN,27,17,17,14-3-0,435,648,67.1,5250,...,77.6,26,188,3.9,7.51,7.93,4.0,4.0,MahoPa00,2022
2,Justin Herbert,LAC,24,17,17,10-7-0,477,699,68.2,4739,...,58.2,38,206,5.2,6.15,6.22,4.0,5.0,HerbJu00,2022
3,Tom Brady,TAM,45,17,17,8-9-0,490,733,66.8,4694,...,52.5,22,160,2.9,6.01,6.13,4.0,5.0,BradTo00,2022
4,Kirk Cousins*,MIN,34,17,17,13-4-0,424,643,65.9,4547,...,49.9,46,329,6.7,6.12,6.05,8.0,8.0,CousKi00,2022
5,Joe Burrow*,CIN,26,16,16,12-4-0,414,606,68.3,4475,...,58.7,41,259,6.3,6.52,6.76,3.0,4.0,BurrJo01,2022
6,Jared Goff*,DET,28,17,17,9-8-0,382,587,65.1,4438,...,61.1,23,156,3.8,7.02,7.45,3.0,3.0,GoffJa00,2022
7,Josh Allen*,BUF,26,16,16,13-3-0,359,567,63.3,4283,...,71.4,33,162,5.5,6.87,6.99,3.0,4.0,AlleJo02,2022
8,Geno Smith*,SEA,32,17,17,9-8-0,399,572,69.8,4282,...,60.8,46,348,7.4,6.37,6.54,2.0,3.0,SmitGe00,2022
9,Trevor Lawrence*,JAX,23,17,17,9-8-0,387,584,66.3,4113,...,54.5,27,184,4.4,6.43,6.66,3.0,2.0,LawrTr00,2022


In [13]:
df1 = pd.read_csv('2022.csv',index_col=[0])
df1['year'] = '2022'
df1 =df.drop(columns=['Pos'])
df1

,Player,Tm,Age,G,GS,QBrec,Cmp,Att,Cmp%,Yds,...,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,Player-additional,year
Rk,,,,,,,,,,,,,,,,,,,,,
1,Patrick Mahomes*+,KAN,27,17,17,14-3-0,435,648,67.1,5250,...,77.6,26,188,3.9,7.51,7.93,4.0,4.0,MahoPa00,2022
2,Justin Herbert,LAC,24,17,17,10-7-0,477,699,68.2,4739,...,58.2,38,206,5.2,6.15,6.22,4.0,5.0,HerbJu00,2022
3,Tom Brady,TAM,45,17,17,8-9-0,490,733,66.8,4694,...,52.5,22,160,2.9,6.01,6.13,4.0,5.0,BradTo00,2022
4,Kirk Cousins*,MIN,34,17,17,13-4-0,424,643,65.9,4547,...,49.9,46,329,6.7,6.12,6.05,8.0,8.0,CousKi00,2022
5,Joe Burrow*,CIN,26,16,16,12-4-0,414,606,68.3,4475,...,58.7,41,259,6.3,6.52,6.76,3.0,4.0,BurrJo01,2022
6,Jared Goff*,DET,28,17,17,9-8-0,382,587,65.1,4438,...,61.1,23,156,3.8,7.02,7.45,3.0,3.0,GoffJa00,2022
7,Josh Allen*,BUF,26,16,16,13-3-0,359,567,63.3,4283,...,71.4,33,162,5.5,6.87,6.99,3.0,4.0,AlleJo02,2022
8,Geno Smith*,SEA,32,17,17,9-8-0,399,572,69.8,4282,...,60.8,46,348,7.4,6.37,6.54,2.0,3.0,SmitGe00,2022
9,Trevor Lawrence*,JAX,23,17,17,9-8-0,387,584,66.3,4113,...,54.5,27,184,4.4,6.43,6.66,3.0,2.0,LawrTr00,2022


In [ ]:
df2 = pd.read_csv('2021.csv',index_col=[0])
df2['year'] = '2021'
df2 =df.drop(columns=['Pos'])
df2

In [ ]:
df3 = pd.read_csv('2020.csv',index_col=[0])
df3['year'] = '2020'
df3 =df.drop(columns=['Pos'])
df3

In [ ]:
df4 = pd.read_csv('2019.csv',index_col=[0])
df4['year'] = '2019'
df4 =df.drop(columns=['Pos'])
df4

In [ ]:
df5 = pd.read_csv('2018.csv',index_col=[0])
df5['year'] = '2018'
df5 =df.drop(columns=['Pos'])
df5

In [ ]:
df6 = pd.read_csv('2017.csv',index_col=[0])
df6['year'] = '2017'
df6 =df.drop(columns=['Pos'])
df6

In [ ]:
df7 = pd.read_csv('2016.csv',index_col=[0])
df7['year'] = '2016'
df7 =df.drop(columns=['Pos'])
df7

In [ ]:
df8 = pd.read_csv('2015.csv',index_col=[0])
df8['year'] = '2015'
df8 =df.drop(columns=['Pos'])
df8

In [ ]:
df9 = pd.read_csv('2014.csv',index_col=[0])
df9['year'] = '2014'
df9 =df.drop(columns=['Pos'])
df9

In [ ]:
df10 = pd.read_csv('2013.csv',index_col=[0])
df10['year'] = '2013'
df10 =df.drop(columns=['Pos'])
df10

In [ ]:
df11 = pd.read_csv('2012.csv',index_col=[0])
df11['year'] = '2012'
df11 =df.drop(columns=['Pos'])
df11

In [ ]:
df12 = pd.read_csv('2011.csv',index_col=[0])
df12['year'] = '2011'
df12 =df.drop(columns=['Pos'])
df12

In [ ]:
df13 = pd.read_csv('2010.csv',index_col=[0])
df13['year'] = '2010'
df13 =df.drop(columns=['Pos'])
df13

In [14]:
master_df_new_data = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11, df12,df13])

NameError: name 'df1' is not defined